In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from dotenv import load_dotenv

load_dotenv()

import os

api_key = os.getenv("ANTHROPIC_API_KEY")

if not api_key:
    raise ValueError("Anthropic API key not found")


In [31]:
import os
from jinja2 import Environment, FileSystemLoader

system_prompt_env = Environment(loader=FileSystemLoader("/Users/khannguyen/projects/agent-coding-template/src/agent_coding_template/prompts"))
usecase_prompt_env = Environment(loader=FileSystemLoader("/Users/khannguyen/projects/agent-coding-template/use-case-tests/nvidia-eval-adaptors"))


In [10]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from agent_coding_template.tools import *

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-5-sonnet-20241022")
tools = [DirectoryMapperTool(), FileReaderTool(), ReadmeReaderTool()]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=template.render(repo_dir="/Users/khannguyen/projects/evaluations/ToolTalk"))]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content=[{'text': "I'll help you understand what this code repository is about. Let me check the README file first to get an overview.", 'type': 'text'}, {'id': 'toolu_01CUb6jA6LZXmf2cbQJzHPcT', 'input': {'directory': '/Users/khannguyen/projects/evaluations/ToolTalk'}, 'name': 'ReadmeReaderTool', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01395U2oboJeCunX8TGYPc4i', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 841, 'output_tokens': 97}}, id='run-6111dd98-2c95-4671-bdd7-fac4bbb21f3c-0', tool_calls=[{'name': 'ReadmeReaderTool', 'args': {'directory': '/Users/khannguyen/projects/evaluations/ToolTalk'}, 'id': 'toolu_01CUb6jA6LZXmf2cbQJzHPcT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 841, 'output_tokens': 97, 'total_tokens': 938, 'input_token_details': {'cache_read': 0, 'cache_creati

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Tell me how you would adapt this code to a programmatic interface that I defined.")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': 'Let me check the repository structure to find the relevant implementation files that you would need to adapt.', 'type': 'text'}, {'id': 'toolu_01Ls3Miae7Zj3y1FK4vBAM3g', 'input': {'directory': '/Users/khannguyen/projects/evaluations/ToolTalk'}, 'name': 'DirectoryMapperTool', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01LpEU2A3h3EoKMFWTuVGAPM', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 2734, 'output_tokens': 92}}, id='run-f4cb54fa-6298-4e38-a880-4ff6164827c3-0', tool_calls=[{'name': 'DirectoryMapperTool', 'args': {'directory': '/Users/khannguyen/projects/evaluations/ToolTalk'}, 'id': 'toolu_01Ls3Miae7Zj3y1FK4vBAM3g', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2734, 'output_tokens': 92, 'total_tokens': 2826, 'input_token_details': {'cache_read': 0, 'cache_cre

In [13]:
print(chunk['agent']['messages'][-1].content)

Based on the code examination, here's how you would adapt ToolTalk to your programmatic interface:

1. Create a Custom Predictor:
The main integration point is through the `BaseAPIPredictor` class. You would need to create your own predictor class that inherits from this base class:

```python
class YourCustomPredictor(BaseAPIPredictor):
    def __init__(self, function_docs: List[dict], *args, **kwargs):
        # Initialize your model/interface here
        self.your_interface = YourInterface(*args, **kwargs)
        self.function_docs = function_docs

    def predict(self, metadata: dict, conversation_history: dict) -> dict:
        # Implement your prediction logic here
        # Must return a dict with the following structure:
        # For API calls:
        # {
        #     "role": "api",
        #     "request": {
        #         "api_name": "name_of_api",
        #         "parameters": {param_dict}
        #     }
        # }
        # For assistant responses:
        # {
 

In [40]:
from typing import Dict, TypedDict, Annotated, Sequence
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END, START
from agent_coding_template.tools import *
import uuid

# Define state schema
class AgentState(TypedDict):
    messages: Sequence[HumanMessage]
    thread_id: str

# Create tools and model
model = ChatAnthropic(model_name="claude-3-5-sonnet-20241022", max_tokens=8000)
tools = [DirectoryMapperTool(), FileReaderTool(), ReadmeReaderTool(), FileEditorTool(), CreateDirectoryTool(), FileCopyTool(), FileRenameTool()]

memory = MemorySaver()

system_prompt = template = system_prompt_env.get_template("coding_agent_system_prompt.j2").render()
print(system_prompt)


agent = create_react_agent(model, tools, checkpointer=memory, prompt=system_prompt)

# Load the template file
# template = usecase_prompt_env.get_template("1_add_dependencies.j2")
template = usecase_prompt_env.get_template("2_adaptor_bootstrap.j2")

content = template.render(repo_dir="/Users/khannguyen/projects/evaluations/ToolTalk", adaptor_instruction_filepath="/Users/khannguyen/projects/agent-coding-template/use-case-tests/nvidia-eval-adaptors/nvidia_eval_adaptor_instruction.md")
print(content)

config = {"configurable": {"thread_id": uuid.uuid4()}}

# Stream results
for chunk in agent.stream(
    {"messages": [HumanMessage(content=content)]}, config
):
    print(chunk)
    print("----")

**Code Assistant Protocol**

You are an advanced code editing assistant.

**File Editing Protocol** (MUST FOLLOW when making changes):
1. **Pre-Edit Verification** - ALWAYS use FileReaderTool to:
   - Cross-validate paths using DirectoryMapperTool
   - Get current file state WITH LINE NUMBERS
   - Verify exact location for modifications
   
2. **Modification Rules**:
   - Use FileEditorTool to make changes to the file.
   - NEVER assume line numbers - ALWAYS verify via FileReaderTool
   - NEVER include unchanged content - only send NEW/CHANGED lines
   - Example: To add 1 line at position 42:
     ✅ Correct: insert at line 42 with content
     ❌ Wrong: Rewriting entire file with new line added

**Token Efficiency Guidelines**:
- Use precise parameters in ALL tool calls to avoid unnecessary tokens

**General Operational Guidelines**:
- Ask clarification questions if requirements are ambiguous
# Task: Implement NVIDIA Evaluation Adaptor

## Objective
Create an adaptor that integrates the